In [2]:
import pandas as pd
import numpy as np
import pickle
import requests
from geopy.distance import geodesic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import gradio as gr

# Load dataset
df = pd.read_csv('apartments_data_enriched_with_new_features.csv')

# Load stops dataset
stops_df = pd.read_csv('stops.csv', sep=',', encoding='utf-8')

# Swiss geolocation API base URL
BASE_URL = "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Function to get geolocation data
def get_geolocation(address):
    params = {
        "searchText": address,
        "origins": "address",
        "type": "locations"
    }
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        results = data.get("results", [])
        if results:
            location = results[0]["attrs"]
            return location.get("lat"), location.get("lon")
    
    return None, None  # Default values if no result

# Function to find the closest stop
def get_closest_stop(lat, lon):
    if lat is None or lon is None:
        return "Unknown"  # Default if geolocation fails
    
    # Compute distances to all stops
    stops_df["distance"] = stops_df.apply(lambda row: geodesic((lat, lon), (row["stop_lat"], row["stop_lon"])).meters, axis=1)
    
    # Find the nearest stop
    closest_stop = stops_df.loc[stops_df["distance"].idxmin(), "stop_name"]
    return closest_stop

# Preprocess data (adjusted column names)
features = ['rooms', 'area', 'luxurious', 'zurich_city']  # Adjusted column names

df['luxurious'] = df['luxurious'].astype(int)
df['zurich_city'] = df['zurich_city'].astype(int)

target = 'price'  # Adjusted target column name

# Splitting data
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Save the trained model
with open("trained_apartment_model.pkl", "wb") as f:
    pickle.dump((model, scaler), f)

# Function for Gradio prediction
def predict_price(address, num_rooms, area, luxurious, zurich_city):
    with open("trained_apartment_model.pkl", "rb") as f:
        model, scaler = pickle.load(f)
    
    # Get geolocation data
    lat, lon = get_geolocation(address)
    
    # Get closest stop
    closest_stop = get_closest_stop(lat, lon)
    
    # Prepare input features
    input_data = np.array([[num_rooms, area, int(luxurious), int(zurich_city)]])
    input_data_scaled = scaler.transform(input_data)
    predicted_price = model.predict(input_data_scaled)[0]
    
    return f"CHF {predicted_price:,.2f}", closest_stop

# Create Gradio interface
demo = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Textbox(label="Address"),
        gr.Number(label="Number of rooms"),
        gr.Number(label="Area in m²"),
        gr.Checkbox(label="Luxurious"),
        gr.Checkbox(label="Zurich City"),
    ],
    outputs=[
        gr.Textbox(label="Estimated Price"),
        gr.Textbox(label="Closest Stop")
    ],
    examples=[
        ["Zürcherstrasse 1, 8173 Neerach", 3.5, 65, False, False],
        ["Badenerstrasse 123, 8004 Zürich", 4, 98, False, True],
        ["Robert-Stephenson-Weg 47, 8004 Zürich", 4.5, 148, False, True]
    ],
    title="Apartment Price Predictor",
    description="Predict the price of an apartment in Zurich based on its location and features.",
    article="Expected result for examples: 1850, 3491, 6820"
)

demo.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


c:\Users\ilias\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\ilias\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
